In [ ]:
import pandas as pd
from numpy.linalg import norm
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error,recall_score,ndcg_score
from sklearn.model_selection import ShuffleSplit
from torch_geometric import data

In [ ]:
'''
np.corrcoef
Return Pearson product-moment correlation coefficients.
'''

In [ ]:
def Average(lst): 
	return sum(lst) / len(lst) 

In [ ]:
user_book = np.load('Douban_book/user_book.npy')

X = user_book[:,:2]
y = user_book[:,-1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)

In [18]:
user_movie = np.load('Movielens/user_movie.npy')

X = user_movie[:,:2]
y = user_movie[:,-1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)

In [19]:
#User-based CF
RMSE_M = []
recall_M = []
NDCG_M = []
for i, (train_index, test_index) in enumerate(rs.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

    user_rating = np.zeros((943, 1682), dtype=int)
    for i in range(y_train.shape[0]):
        user, book = X_train[i]
        user_rating[user-1, book-1] = y_train[i]
    
    UCF_cs = np.corrcoef(user_rating)

    predict = []
    for i in range(X_test.shape[0]):
        user, book =  X_test[i]
        most_similar_user = np.argpartition(UCF_cs[user-1],-6)[-6:-1]
        pred = user_rating[most_similar_user,book-1].sum()/5
        predict.append(pred)
    
    RMSE = round(mean_squared_error(np.array(predict),y_test, squared=False),4)
    RMSE_M.append(RMSE)
    recall = round(recall_score(np.round(np.array(predict)),y_test, average='micro'),4)
    recall_M.append(recall)
    NDCG = round(ndcg_score(np.array([predict]),y_test.reshape(1, -1), k=10),4)
    NDCG_M.append(NDCG) 
    print(f"RMSE: {RMSE}, Recall: {recall}, NDCG: {NDCG}")
print(f"Average RMSE: {Average(RMSE_M)}, Average Recall: {Average(recall_M)}, Average NDCG: {Average(NDCG_M)}")    


RMSE: 2.589, Recall: 0.0864, NDCG: 0.3492
RMSE: 2.5827, Recall: 0.0888, NDCG: 0.3562
RMSE: 2.5829, Recall: 0.0822, NDCG: 0.3533
RMSE: 2.5605, Recall: 0.0871, NDCG: 0.3653
RMSE: 2.5796, Recall: 0.0919, NDCG: 0.3546
Average RMSE: 2.5789400000000002, Average Recall: 0.08728, Average NDCG: 0.35572


In [ ]:
# 扣除所有使用者的加總平均，undo
mean_by_user = user_rating.sum(axis=1)/np.count_nonzero(user_rating, axis=1)
mean_by_user
none_zero = np.where(user_rating > 0)
none_zero[]

In [ ]:
UFC = np.load('cs_user_rating.npy')
UFC